In [3]:
!pip install openai


  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [83 lines of output]
      /opt/conda/lib/python3.11/site-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully.
        │ exit code: 1
        ╰─> [17 lines of output]
            Traceback (most recent call last):
              File "<string>",

In [ ]:
import os
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")
#print(f"API Key:{api_key}")
openai_client = OpenAI(
    base_url="https://api.openai.com/v1",
    api_key= #enter api key
)



def invoke_openai(prompt, model = 'gpt-4o-mini', json_mode = False):
    try:
        kwargs = {
                "model": model,
                "messages": [{"role": "user", "content": prompt}]
            }

        if json_mode:
            kwargs["response_format"] = {"type": "json_object"}

        response = openai_client.chat.completions.create(**kwargs)

        return response.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")

In [12]:
import pandas as pd

In [13]:
# get the csv file
file_path = 'student_profiles.csv'

# read the csv file
df = pd.read_csv(file_path)

In [17]:
row1 = df.iloc[0]


In [45]:
prompt = """
<|im_start|>system
Evaluate the student profile objectively using this JSON schema:

<schema>
{{
    "overall_score": "number",
    "category_scores": {{
        "Academics_and_research": "number",
        "professional_experience": "number", 
        "Leadership and Initiative": "number",
        "depth_of_experience": "number"
    }}
}}
</schema>

Scoring Guidelines:
- Maximum typical score is 100
- Scores above 85 are EXTREMELY rare
- 50 represents a solid, competitive student profile
- Scoring criteria:
  - Be more critical and conservative
  - Scrutinize each achievement's substantiveness
  - Avoid grade inflation
  - Emphasize quality over quantity

Scoring Constraints:
- Rarely award more than 85 points
- Most students should score between 45-60
- Exceptional achievements warrant careful consideration
- Penalize vague or unsubstantiated claims
<|im_end|>

<|im_start|>user
Evaluate the following {major} student profile:
{student_profile}
<|im_end|>

<|im_start|>assistant
"""

In [50]:
prompt_formatted = prompt.format(major=row1['major'], student_profile=row1['bio'])
response = invoke_openai(prompt_formatted, json_mode=True)

In [51]:
import json
response_json = json.loads(response)

response_json

{'overall_score': 60,
 'category_scores': {'Academics_and_research': 55,
  'professional_experience': 60,
  'Leadership and Initiative': 65,
  'depth_of_experience': 60}}